# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [29]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("cities.csv")
cities_df

,Unnamed: 0,City,Temperature,Latitude,Longitude,Cloudiness,Wind_Speed,Humidity
0,0,rikitea,25.04,-23.1203,-134.9692,15,3.09,76
1,1,broome,7.46,42.2506,-75.8330,26,1.79,54
2,2,atar,35.20,20.5169,-13.0499,23,6.59,11
3,3,mudgal,27.31,16.0167,76.4333,50,10.57,47
4,4,bathsheba,29.00,13.2167,-59.5167,40,8.23,54
...,...,...,...,...,...,...,...,...
542,542,zapolyarnyy,1.00,69.4154,30.8136,40,2.06,80
543,543,taltal,19.30,-25.4000,-70.4833,16,1.83,68
544,544,pangnirtung,2.00,66.1451,-65.7125,90,2.06,87
545,545,dipalpur,30.56,30.6709,73.6529,1,2.71,15


In [3]:
del cities_df['Unnamed: 0']
cities_df

,City,Temperature,Latitude,Longitude,Cloudiness,Wind_Speed,Humidity
0,rikitea,25.04,-23.1203,-134.9692,15,3.09,76
1,broome,7.46,42.2506,-75.8330,26,1.79,54
2,atar,35.20,20.5169,-13.0499,23,6.59,11
3,mudgal,27.31,16.0167,76.4333,50,10.57,47
4,bathsheba,29.00,13.2167,-59.5167,40,8.23,54
...,...,...,...,...,...,...,...
542,zapolyarnyy,1.00,69.4154,30.8136,40,2.06,80
543,taltal,19.30,-25.4000,-70.4833,16,1.83,68
544,pangnirtung,2.00,66.1451,-65.7125,90,2.06,87
545,dipalpur,30.56,30.6709,73.6529,1,2.71,15


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [39]:
# Configure gmaps
gmaps.configure(api_key=g_key) 

# Use the Lat and Lng as locations and Humidity as the weight
locations = cities_df[["Latitude", "Longitude"]]
humidity = cities_df["Humidity"]

,Latitude,Longitude
0,-23.1203,-134.9692
1,42.2506,-75.8330
2,20.5169,-13.0499
3,16.0167,76.4333
4,13.2167,-59.5167
...,...,...
542,69.4154,30.8136
543,-25.4000,-70.4833
544,66.1451,-65.7125
545,30.6709,73.6529


In [13]:
# Add Heatmap layer to map
fig = gmaps.figure(center = (0,0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,  max_intensity=100, point_radius=2, dissipating=False)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
   * A max temperature lower than 80 degrees but higher than 70.
   * Wind speed less than 10 mph.
   * Zero cloudiness.
   * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
* Drop any rows will null values.

In [24]:
#Narrow down the cities to fit weather conditions.
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.

temp = cities_df['Temperature']
windy = cities_df['Wind_Speed']
cloudy = cities_df['Cloudiness']

#A max temperature lower than 80 degrees but higher than 70.
# I used metric when I created the cities data, so I"m converting the temperatures to the corresponding values in Celsius
conditions = cities_df.loc[(temp > 21.1) & (temp < 26.67)]

#Wind speed less than 10 mph.
conditions = conditions.loc[(windy < 10)]

#Zero cloudiness.
conditions = conditions.loc[(cloudy == 0)]

conditions


,City,Temperature,Latitude,Longitude,Cloudiness,Wind_Speed,Humidity
160,pozo colorado,26.32,-23.4923,-58.7972,0,3.09,27
243,san vicente,21.96,-26.6167,-54.1333,0,1.96,34
389,abnub,25.00,27.2696,31.1511,0,6.17,41
429,nouadhibou,22.00,20.9310,-17.0347,0,9.26,64
476,pokhara,22.48,28.2333,83.9833,0,3.37,20
507,ilo,22.00,-17.6394,-71.3375,0,5.14,68
526,romitan,23.00,39.9343,64.3814,0,3.09,27


In [25]:
#Drop any rows will null values
conditions.dropna()

,City,Temperature,Latitude,Longitude,Cloudiness,Wind_Speed,Humidity
160,pozo colorado,26.32,-23.4923,-58.7972,0,3.09,27
243,san vicente,21.96,-26.6167,-54.1333,0,1.96,34
389,abnub,25.00,27.2696,31.1511,0,6.17,41
429,nouadhibou,22.00,20.9310,-17.0347,0,9.26,64
476,pokhara,22.48,28.2333,83.9833,0,3.37,20
507,ilo,22.00,-17.6394,-71.3375,0,5.14,68
526,romitan,23.00,39.9343,64.3814,0,3.09,27


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
#Add a "Hotel Name" column to the DataFrame
conditions['Hotel Name'] = ''
conditions

,City,Temperature,Latitude,Longitude,Cloudiness,Wind_Speed,Humidity,Hotel Name
160,pozo colorado,26.32,-23.4923,-58.7972,0,3.09,27,
243,san vicente,21.96,-26.6167,-54.1333,0,1.96,34,
389,abnub,25.00,27.2696,31.1511,0,6.17,41,
429,nouadhibou,22.00,20.9310,-17.0347,0,9.26,64,
476,pokhara,22.48,28.2333,83.9833,0,3.37,20,
507,ilo,22.00,-17.6394,-71.3375,0,5.14,68,
526,romitan,23.00,39.9343,64.3814,0,3.09,27,


In [28]:
#Store into variable named hotel_df

hotel_df = conditions
hotel_df

,City,Temperature,Latitude,Longitude,Cloudiness,Wind_Speed,Humidity,Hotel Name
160,pozo colorado,26.32,-23.4923,-58.7972,0,3.09,27,
243,san vicente,21.96,-26.6167,-54.1333,0,1.96,34,
389,abnub,25.00,27.2696,31.1511,0,6.17,41,
429,nouadhibou,22.00,20.9310,-17.0347,0,9.26,64,
476,pokhara,22.48,28.2333,83.9833,0,3.37,20,
507,ilo,22.00,-17.6394,-71.3375,0,5.14,68,
526,romitan,23.00,39.9343,64.3814,0,3.09,27,


In [30]:
#Set parameters to search for hotels with 5000 meters.

# geocoordinates
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

In [41]:
#Hit the Google Places API for each city's coordinates.

for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    long = row["Longitude"]
    
    params["location"] = f"{lat},{long}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params)
    
    response = response.json()
    
    hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]


# base url
#base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
#response = requests.get(base_url, params=params)

hotel_df


,City,Temperature,Latitude,Longitude,Cloudiness,Wind_Speed,Humidity,Hotel Name
160,pozo colorado,26.32,-23.4923,-58.7972,0,3.09,27,Villa Hayes
243,san vicente,21.96,-26.6167,-54.1333,0,1.96,34,San Pedro
389,abnub,25.00,27.2696,31.1511,0,6.17,41,El-Hamam
429,nouadhibou,22.00,20.9310,-17.0347,0,9.26,64,Nouadhibou
476,pokhara,22.48,28.2333,83.9833,0,3.37,20,Pokhara
507,ilo,22.00,-17.6394,-71.3375,0,5.14,68,Ilo District
526,romitan,23.00,39.9343,64.3814,0,3.09,27,Ramitan


In [ ]:
#rename columns to fit code below



In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
